In [1]:
from EM_part import *

In [2]:
DATA_PATH = "./Data_FD003/preprocessed data/"
attribute = ['Unit', 'T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30',
             'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']
df_train = pd.read_csv(DATA_PATH + 'TD_data.csv', names=attribute, header=None)
df_test = pd.read_csv(DATA_PATH + 'Test_data.csv', names=attribute, header=None)

df_train = Preprocessing.add_timeseries(df_train)
df_test = Preprocessing.add_timeseries(df_test)

train_label=pd.read_csv(DATA_PATH +"TD_mode.csv", header=None).values
test_label=pd.read_csv(DATA_PATH +"Test_mode.csv", header=None).values

In [3]:
train_dataset=AircraftDataset(df_train,train_label) # 不插0计算创建dataset的子类
test_dataset = AircraftDataset(df_test,test_label)

train_dataset_mode_a =  AircraftDataset_one_mode(df_train,train_label,-1)
train_dataset_mode_b =  AircraftDataset_one_mode(df_train,train_label,1)

In [4]:
train_dataset_mode_a[0]["mode"],train_dataset_mode_b[0]["mode"]

(array([1., 0.]), array([0., 1.]))

In [19]:
set_seed(613)

pi_k=np.random.random(size=None)
w_k=np.random.rand(6,1)
mu_k=np.random.rand(3,1)
Sigma_k=generate_rand_psd(3)
sigma_k_2=np.random.random(size=None)

pi_j=np.random.random(size=None)
w_j=np.random.rand(6,1)
mu_j=np.random.rand(3,1)
Sigma_j=generate_rand_psd(3)
sigma_j_2=np.random.random(size=None)

# w_2=0.48
# pi_2=np.random.random(size=None)
# mu_2=np.random.rand(3,1)
# Sigma_2=generate_rand_psd(3)
# sigma_2_2=np.random.random(size=None)

In [7]:
def get_hat_rho_l(x_l,Phi_l,*arg
#                   [pi_k, w_k,  mu_k, Sigma_k, sigma_k_2],
#                      [pi_k, w_k, mu_k, Sigma_k, sigma_k_2]
                 ):
    num_mode=len(arg)
    ln_numerator_list=[]
    for idx in range(num_mode):
        [pi_k, w_k,  mu_k, Sigma_k, sigma_k_2] = arg[idx]
        ln_numerator_list.append(ln_get_hat_rho_k_numerator(x_l, Phi_l, pi_k,
                                                            w_k,  mu_k,
                                                            Sigma_k, sigma_k_2))
    sorted_list=np.sort(ln_numerator_list)[::-1]
    if np.abs(sorted_list[0]-sorted_list[-1])>5:
        max_idx = np.argmax(ln_numerator_list)
        hat_rho_l = np.zeros((num_mode,1))
        hat_rho_l[max_idx] = 1
        return hat_rho_l
    else:
        hat_rho_l_list=[]
        for idx in range(num_mode):
            denominator = 0
            for idx_ in range(num_mode):
                denominator+=np.exp(ln_numerator_list[idx_]-ln_numerator_list[idx])
            hat_rho_l_list.append(1 / denominator.item())
        return np.array(hat_rho_l_list).reshape(num_mode,1)

get_hat_rho_l(x_l,Phi_l,
                 [pi_k, w_k,  mu_k, Sigma_k, sigma_k_2]
                      
                 )

array([[1.]])

In [13]:
inv=32
x_l=train_dataset_mode_b[inv]["input"]
Phi_l=train_dataset_mode_b[inv]["Phi_l"]

In [22]:
instance = train_dataset_mode_b
arg = [[pi_j, w_j,  mu_j, Sigma_j, sigma_j_2]]
num_mode = len(arg)

In [51]:
old_w = []
old_mu = []
old_Sigma = []
old_sigma_2 = []
for idx in range(num_mode):
    old_w.append(arg[idx][1])
    old_mu.append(arg[idx][2])
    old_Sigma.append(arg[idx][3])
    old_sigma_2.append(arg[idx][4])
old_w = np.array(old_w)
old_mu = np.array(old_mu)
old_Sigma = np.array(old_Sigma)
old_sigma_2 = np.array(old_sigma_2)

updated_w = get_new_w_k_list(instance, *arg)
for idx in range(num_mode):
    arg[idx][1] = updated_w[idx]

updated_pi_k = get_N_k_and_hat_pi_k(instance, *arg)[1]
updated_mu = get_new_mu_list(instance, *arg)  # num_mode*3*1
updated_Sigma = get_new_Sigma_K_list(instance, *arg)
updated_sigma_2 = get_new_sigma_k_2_list(instance, *arg)

loss_w = (np.square(old_w - updated_w)).mean().item()
loss_mu = (np.square(old_mu - updated_mu)).mean().item()
loss_Sigma = (np.square(old_Sigma - updated_Sigma)).mean().item()
loss_sigma_2 = (np.square(old_sigma_2 - updated_sigma_2)).mean().item()

for idx in range(num_mode):
    arg[idx][0] = updated_pi_k[idx]
    arg[idx][2] = updated_mu[idx]
    arg[idx][3] = updated_Sigma[idx]
    arg[idx][4] = updated_sigma_2[idx]

In [56]:
arg2=arg[0]
arg2[-1]=np.array([20])
arg2

[array([1.]),
 array([[-0.26277003],
        [ 3.36193867],
        [-0.43534186],
        [-2.91294644],
        [ 2.89379109],
        [-3.03367739]]),
 array([[ 0.60444151],
        [-0.0452041 ],
        [ 0.00241464]]),
 array([[ 9.34432394e-03, -1.23105183e-03,  3.70596317e-05],
        [-1.23105183e-03,  3.19993143e-04, -3.02091035e-06],
        [ 3.70596317e-05, -3.02091035e-06,  6.73125327e-06]]),
 array([20])]

In [6]:
arg1=EM_iteration(train_dataset_mode_a, 100, 0.001, [pi_k, w_k,  mu_k, Sigma_k, sigma_k_2])

converge!, itr=8


In [21]:
arg2=EM_iteration(train_dataset_mode_b, 100, 0.001, [pi_j, w_j,  mu_j, Sigma_j, sigma_j_2])

itr=10,loss=4438.547931222325
itr=20,loss=3.734894011359676e+29


KeyboardInterrupt: 

In [57]:
np.save('./numpy/arg1.npy',np.array(arg1,dtype=object).reshape(-1))
np.save('./numpy/arg2.npy',np.array(arg2,dtype=object).reshape(-1))

In [58]:
arg_1=[np.load('./numpy/arg1.npy',allow_pickle=True)[i] for i in range(5)]
arg_2=[np.load('./numpy/arg2.npy',allow_pickle=True)[i] for i in range(5)]
arg_1[0]=np.array([0.53])
arg_2[0]=np.array([0.47])

In [59]:
arg_1,arg_2

([array([0.53]),
  array([[ 0.05035139],
         [ 0.01534575],
         [ 0.00103662],
         [-0.11407601],
         [ 0.15346991],
         [-0.43868717]]),
  array([[ 0.48909126],
         [-0.02458209],
         [ 0.00272191]]),
  array([[ 5.73567785e-03, -1.14175009e-03,  5.12084317e-05],
         [-1.14175009e-03,  3.64451713e-04, -1.23774971e-05],
         [ 5.12084317e-05, -1.23774971e-05,  5.72251389e-06]]),
  array([18.72508017])],
 [array([0.47]),
  array([[-0.26277003],
         [ 3.36193867],
         [-0.43534186],
         [-2.91294644],
         [ 2.89379109],
         [-3.03367739]]),
  array([[ 0.60444151],
         [-0.0452041 ],
         [ 0.00241464]]),
  array([[ 9.34432394e-03, -1.23105183e-03,  3.70596317e-05],
         [-1.23105183e-03,  3.19993143e-04, -3.02091035e-06],
         [ 3.70596317e-05, -3.02091035e-06,  6.73125327e-06]]),
  array([20])])

In [63]:
idx=3
instance=train_dataset_mode_b
x_q = instance[idx]["input"]
Phi_q = instance[idx]["Phi_l"]
get_hat_rho_l(x_q, Phi_q, arg_1,arg_2),instance[idx]['mode']

(array([[1.],
        [0.]]),
 array([0., 1.]))

In [ ]:
EM_iteration(train_dataset, 100, 1, arg_1,arg_2)

In [ ]:
arg_1,arg_2

In [2]:
print("nbnbnbnb")

nbnbnbnb
